**Set environment**

In [1]:
source ../config/config_duke.sh
show_env

You are on Duke Server: HARDAC
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code
SING DIRECTORY (FD_SING): /data/reddylab/Kuei/singularity
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log



**Check data**

In [2]:
ASSAY=A001_K562_WSTARRseq
FD_INP=${FD_RES}/results/${ASSAY}/fragment_count
FP_INPS=($(ls ${FD_INP}/*WGS*bed.gz))

echo Total: ${#FP_INPS[@]}
for FP_INP in ${FP_INPS[@]}; do
    echo $(basename ${FP_INP})
done

Total: 7
A001_K562_WSTARRseq.Input.rep1.WGS.unstranded.bed.gz
A001_K562_WSTARRseq.Input.rep2.WGS.unstranded.bed.gz
A001_K562_WSTARRseq.Input.rep3.WGS.unstranded.bed.gz
A001_K562_WSTARRseq.Input.rep4.WGS.unstranded.bed.gz
A001_K562_WSTARRseq.Output.rep1.WGS.unstranded.bed.gz
A001_K562_WSTARRseq.Output.rep2.WGS.unstranded.bed.gz
A001_K562_WSTARRseq.Output.rep3.WGS.unstranded.bed.gz


## RUN

In [3]:
sbatch -p ${NODE} \
    --exclude=dl-01 \
    --cpus-per-task 8 \
    --mem 8G \
    --output ${FD_LOG}/coverage_library_size_WSTARR.%a.txt \
    --array 0 \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config/config_duke.sh
ASSAY=A001_K562_WSTARRseq

### print start message
timer_start=`date +%s`
echo "Hostname:          " $(hostname)
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### set input and output files
FD_INP=${FD_RES}/results/${ASSAY}/fragment_count
FP_INPS=($(ls ${FD_INP}/*WGS*bed.gz))

FD_OUT=${FD_RES}/results/${ASSAY}/coverage/summary
FN_OUT=library_size_count.csv
FP_OUT=${FD_OUT}/${FN_OUT}

### init
mkdir -p   ${FD_OUT}
echo  -n > ${FP_OUT}

### get library size
for FP_INP in ${FP_INPS[@]}; do
    
    ### show I/O file
    echo ++++++++++++++++++++++++++++++++++++++
    echo "Input: " ${FP_INP}
    echo
    echo "show first few lines of file"
    zcat ${FP_INP} | head -n 3
    echo
    
    ### count the total counts and append the results to the output file
    FN_INP=$(basename ${FP_INP})
    zcat ${FP_INP} |\
        awk -v OFS=, -v FNAME=${FN_INP} '{sum += $5} END {print FNAME, sum}' \
        >> ${FP_OUT}
done

### show I/O file
echo "Output file: ${FP_OUT}"
echo
echo "Show the lines"
cat ${FP_OUT}
echo

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"
EOF

Submitted batch job 29575241


## Check

In [8]:
cat ${FD_LOG}/coverage_library_size_WSTARR.0.txt

Hostname:           x3-05-2.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         03-12-23+14:07:31

++++++++++++++++++++++++++++++++++++++
Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/A001_K562_WSTARRseq/fragment_count/A001_K562_WSTARRseq.Input.rep1.WGS.unstranded.bed.gz

show first few lines of file
chr1	10335	10549	chr1_10335_10549	1	.
chr1	16983	17500	chr1_16983_17500	1	.
chr1	16985	17499	chr1_16985_17499	1	.

++++++++++++++++++++++++++++++++++++++
Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/A001_K562_WSTARRseq/fragment_count/A001_K562_WSTARRseq.Input.rep2.WGS.unstranded.bed.gz

show first few lines of file
chr1	10244	10370	chr1_10244_10370	1	.
chr1	14494	14961	chr1_14494_14961	1	.
chr1	15593	16258	chr1_15593_16258	1	.

++++++++++++++++++++++++++++++++++++++
Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/A001_K562_WSTARRseq/fragment_count/A001_K562_WSTARRseq.Input.rep3.WGS.unstranded.bed.gz

show first few lines o